In [34]:
import torch
from torch.utils.data import Dataset

from PIL import Image
import os


In [77]:
data_dir = '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/'

import glob
import re
import hashlib



def as_bytes(bytes_or_text, encoding='utf-8'):
    import six as _six
    
    """Converts either bytes or unicode to `bytes`, using utf-8 encoding for text.
    Args:
    bytes_or_text: A `bytes`, `str`, or `unicode` object.
    encoding: A string indicating the charset for encoding unicode.
    Returns:
    A `bytes` object.
    Raises:
    TypeError: If `bytes_or_text` is not a binary or unicode string.
    """
    if isinstance(bytes_or_text, _six.text_type):
        return bytes_or_text.encode(encoding)
    elif isinstance(bytes_or_text, bytes):
        return bytes_or_text
    else:
        raise TypeError('Expected binary or unicode string, got {}'.format(bytes_or_text,))
   


def create_image_lists(data_dir, testing_percentage, validation_percentage):
    MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 -1
    if not os.path.exists(data_dir):
        print("error")
        return None
    result = {}
    sub_dirs = [x[0] for x in os.walk(data_dir)]
    print(sub_dirs)
    is_root_dir = True
    for sub_dir in sub_dirs:
        if is_root_dir:
            is_root_dir = False
            continue
        extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']
        file_list = []
        dir_name = os.path.basename(sub_dir)
        if dir_name == data_dir:
            continue
        for extension in extensions:
            file_glob = os.path.join(data_dir, dir_name, '*' + extension)
            file_list.extend(glob.glob(file_glob))
        if not file_list:
            print("No files found")
            continue
        if len(file_list) < 20:
            print("folder has less than 20 images, this may cause issues")
        if len(file_list) > MAX_NUM_IMAGES_PER_CLASS:
            print("folder has too many images")
        label_name = re.sub(r'[^a-z0-9]+', ' ', dir_name.lower())
        training_images = []
        testing_images = []
        validation_images = []
        for file_name in file_list:
            base_name = os.path.basename(file_name)
            hash_name = re.sub(r'_nohash_.*$', '', file_name)
            hash_name_hashed = hashlib.sha1(as_bytes(hash_name)).hexdigest()
            percentage_hash = ((int(hash_name_hashed, 16) %
                                (MAX_NUM_IMAGES_PER_CLASS + 1)) *
                               (100.0 / MAX_NUM_IMAGES_PER_CLASS))
            if percentage_hash < validation_percentage:
                validation_images.append(base_name)
            elif percentage_hash < (testing_percentage + validation_percentage):
                testing_images.append(base_name)
            else:
                training_images.append(base_name)
            result[label_name] = {
                'dir': dir_name,
                'training': training_images,
                'testing': testing_images,
                'validation': validation_images,
            }
    return result

In [79]:
result = create_image_lists(data_dir, 20, 10)


['/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/RIPPLES', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/OUTCROPS', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/MEGA', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/POCKS', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/ROCK', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/SEA_GRASS', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/DRAGS', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/BLACKS', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/FLATS', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/SANDS', '/home/tim/Documents/Masters/CNN_SB_classifier/datasets/Seabed Images/ROUGHS']


In [88]:
len(result['outcrops']['training'])


1733